<a href="https://colab.research.google.com/github/caseykotowski/Song-Features-Over-Time/blob/main/MCC_Loading_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LoadingData").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Load data from S3 Buckets into Spark DataFrame

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://data-bucket-calixto.s3.us-east-2.amazonaws.com/tracks.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("tracks.csv"), sep=",", header=True)

In [4]:
#confirm commas are separating columns as intended

df.where(df.id=='3hOdlX5u3WGoLjegO9XvRz').show()



+--------------------+--------------------+----------+-----------+--------+--------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|                  id|                name|popularity|duration_ms|explicit|             artists|          id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|
+--------------------+--------------------+----------+-----------+--------+--------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|3hOdlX5u3WGoLjegO...|"Wunderbar - from...|         0|     215467|       0|['Alfred Drake'; ...|['7DDaQqjdIkLxhp1...|        1932|       0.319| 0.263|  8| -12.354|   1|     0.0643|       0.961|               0|   0.343|  0.224|169.047|            

In [5]:
# Show DataFrame
df.show()

+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|                  id|                name|popularity|duration_ms|explicit|            artists|          id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|
+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|35iwgR4jXetI318WE...|               Carve|         6|     126903|       0|            ['Uli']|['45tIt06XoI0Iio4...|   2/22/1922|       0.645| 0.445|  0| -13.338|   1|      0.451|       0.674|           0.744|   0.151|  0.127|104.851|             3|


In [6]:
# Print our schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- id_artists: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)



In [7]:
# Read in artist data from S3 Buckets
from pyspark import SparkFiles
url = "https://data-bucket-calixto.s3.us-east-2.amazonaws.com/artists.csv"
spark.sparkContext.addFile(url)
a_df = spark.read.csv(SparkFiles.get("artists.csv"), sep=",", header=True)

# Show DataFrame
a_df.show()

+--------------------+---------+------+--------------------+----------+
|                  id|followers|genres|                name|popularity|
+--------------------+---------+------+--------------------+----------+
|0DheY5irMjBUeLybb...|        0|    []|Armid & Amir Zare...|         0|
|0DlhY15l3wsrnlfGi...|        5|    []|         ปูนา ภาวิณี|         0|
|0DmRESX2JknGPQyO1...|        0|    []|               Sadaa|         0|
|0DmhnbHjm1qw6NCYP...|        0|    []|           Tra'gruda|         0|
|0Dn11fWM7vHQ3rinv...|        2|    []|Ioannis Panoutsop...|         0|
|0DotfDlYMGqkbzfBh...|        7|    []|       Astral Affect|         0|
|0DqP3bOCiC48L8SM9...|        1|    []|           Yung Seed|         0|
|0Drs3maQb99iRglyT...|        0|    []|               Wi'Ma|         0|
|0DsPeAi1gxPPnYjgp...|        0|    []|             lentboy|         0|
|0DtvnTxgZ9K5YaPS5...|       20|    []|            addworks|         0|
|0Du4t8UK1lBD8TnOI...|        1|    []|         Harrbazzter|    

In [8]:
a_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- followers: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)



In [9]:
a_df = a_df.drop("followers")
a_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)



# Clean up Data

In [10]:
print(df.count())
df.dropna()
print(df.count())
df.dropDuplicates()
print(df.count())

586672
586672
586672


# Replace special characters from Tracks table

In [11]:
import pyspark.sql.functions as F
import re

In [12]:
df = df.withColumn("id_artists", F.regexp_replace(F.regexp_replace("id_artists", "\\['", ""), "\\']", ""))
df.show()

+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|                  id|                name|popularity|duration_ms|explicit|            artists|          id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|
+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|35iwgR4jXetI318WE...|               Carve|         6|     126903|       0|            ['Uli']|45tIt06XoI0Iio4LB...|   2/22/1922|       0.645| 0.445|  0| -13.338|   1|      0.451|       0.674|           0.744|   0.151|  0.127|104.851|             3|


In [13]:
df = df.withColumn("artists", F.regexp_replace(F.regexp_replace("artists", "\\['", ""), "\\']", ""))
df.show()

+--------------------+--------------------+----------+-----------+--------+---------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|                  id|                name|popularity|duration_ms|explicit|        artists|          id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|
+--------------------+--------------------+----------+-----------+--------+---------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|35iwgR4jXetI318WE...|               Carve|         6|     126903|       0|            Uli|45tIt06XoI0Iio4LB...|   2/22/1922|       0.645| 0.445|  0| -13.338|   1|      0.451|       0.674|           0.744|   0.151|  0.127|104.851|             3|
|021ht4sdgPcrDgS

# Remove special characters from Artists table

In [14]:
a_df = a_df.withColumn("genres", F.regexp_replace(F.regexp_replace("genres", "\\['", ""), "\\']", ""))
a_df.show(100)

+--------------------+--------------+------------------------+----------+
|                  id|        genres|                    name|popularity|
+--------------------+--------------+------------------------+----------+
|0DheY5irMjBUeLybb...|            []|    Armid & Amir Zare...|         0|
|0DlhY15l3wsrnlfGi...|            []|             ปูนา ภาวิณี|         0|
|0DmRESX2JknGPQyO1...|            []|                   Sadaa|         0|
|0DmhnbHjm1qw6NCYP...|            []|               Tra'gruda|         0|
|0Dn11fWM7vHQ3rinv...|            []|    Ioannis Panoutsop...|         0|
|0DotfDlYMGqkbzfBh...|            []|           Astral Affect|         0|
|0DqP3bOCiC48L8SM9...|            []|               Yung Seed|         0|
|0Drs3maQb99iRglyT...|            []|                   Wi'Ma|         0|
|0DsPeAi1gxPPnYjgp...|            []|                 lentboy|         0|
|0DtvnTxgZ9K5YaPS5...|            []|                addworks|         0|
|0Du4t8UK1lBD8TnOI...|            []| 

In [15]:
a_df = a_df.withColumn("genres", F.regexp_replace("genres", "\\[]", ""))
a_df.show(100)

+--------------------+--------------+------------------------+----------+
|                  id|        genres|                    name|popularity|
+--------------------+--------------+------------------------+----------+
|0DheY5irMjBUeLybb...|              |    Armid & Amir Zare...|         0|
|0DlhY15l3wsrnlfGi...|              |             ปูนา ภาวิณี|         0|
|0DmRESX2JknGPQyO1...|              |                   Sadaa|         0|
|0DmhnbHjm1qw6NCYP...|              |               Tra'gruda|         0|
|0Dn11fWM7vHQ3rinv...|              |    Ioannis Panoutsop...|         0|
|0DotfDlYMGqkbzfBh...|              |           Astral Affect|         0|
|0DqP3bOCiC48L8SM9...|              |               Yung Seed|         0|
|0Drs3maQb99iRglyT...|              |                   Wi'Ma|         0|
|0DsPeAi1gxPPnYjgp...|              |                 lentboy|         0|
|0DtvnTxgZ9K5YaPS5...|              |                addworks|         0|
|0Du4t8UK1lBD8TnOI...|              | 

# Write DataFrame to RDS

In [16]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-23 04:24:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.84MB/s    in 0.2s    

2021-08-23 04:24:02 (4.84 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [17]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

Enter database password··········


In [18]:
 # Configuration for RDS instance
mode = "append"
jdbc_url="jdbc:postgresql://spotify-db.c3uki6gnymem.us-east-2.rds.amazonaws.com:5432/spotify"
config = {"user":"root", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [19]:
df.write.jdbc(url=jdbc_url, table="track_info", mode="append", properties=config)

In [20]:
a_df.write.jdbc(url=jdbc_url, table="artist_info", mode=mode, properties=config)

# Export to csv

In [21]:
df.toPandas().to_csv('tracks.csv')


In [22]:
a_df.toPandas().to_csv('artists.csv')